In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as pdr
import math
import csv

# Import EDD utils to import study as pandas dataframe
from edd_utils import login, export_study

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from datetime import datetime
import seaborn as sns; sns.set(color_codes=True)
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label, Layout ,Button,AppLayout, jslink, IntText, IntSlider
import matplotlib.colors as mcolors
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly
%matplotlib inline

In [3]:
FILE='DBTL1/DBTL1_Compiled.csv'
df1=pd.read_csv(FILE)
strains = df1['Line Name'].unique()
size=len(strains)
size

192

In [4]:
Line_Name = {}
data = []

#fig, axs = plt.subplots(int(size/3), int(2*(size/3)), figsize=(21, 14),sharey=True, sharex=True) 

#nrow = math.ceil(size/3)
#ncol = 2*(size/3)

for strain in strains:
    Line_Name[strain]=df1[df1['Line Name']==strain]
    
for k, strain in enumerate(strains,start=1):
    nlog=np.log(Line_Name[strain]['Value'])
    time=Line_Name[strain]['Time']

    df1 = pd.concat([time, nlog], axis=1)
    time_range2=np.arange(0,len(df1['Time']))*5
    
    slopes=[]
    r2s=[]
    pvals=[]
    intercepts=[]
    rolling_window = 60
    
    # Perform rolling linear regression
    for i in range(len(df1) - rolling_window + 1):
        subset = df1.iloc[i:i+rolling_window]
        X = sm.add_constant(subset['Time'])  # Add a constant term for the intercept
        y = subset['Value']
        model = sm.OLS(y, X).fit()
        
        slope = model.params['Time']  # Get the slope coefficient
        slopes.append(slope)
        intercept = model.params['const'] # Get the intercept
        intercepts.append(intercept)
        r2=model.rsquared
        r2s.append(r2)
        pval=model.pvalues
        pvals.append(pval)

    max1=max(slopes)
    umax_pos=(slopes.index(max(slopes)))
    pval=pvals[umax_pos]
    r2=r2s[umax_pos]
    intercept=intercepts[umax_pos]
    y_pred = max1*time+intercept
    
    result=(strain, max1, r2)
    print(result)

    data.append(result)
    
#    ax=axs.flatten()[k-1]
#    ax.set_title(strain, size=10)
#    ax.plot(time_range2, nlog)
#    ax.plot(time_range2, y_pred)
#    ax.set_ylim([0, 4])
#    plt.tight_layout()

('PP_0226_PP_0897_PP_5288-R1', 0.10436276133302677, 0.9757301341143518)
('PP_1319_PP_1457_PP_4192-R1', 0.0937140604384871, 0.9734214710971257)
('PP_2213_PP_2471_PP_4266-R1', 0.0956223950254023, 0.9462626300605039)
('PP_3123_PP_4862_PP_5186-R1', 0.10950523292089065, 0.985820426273694)
('PP_2137_PP_4116_PP_4187-R1', 0.11043440539363802, 0.9793811583382883)
('PP_4189_PP_4550_PP_4862-R1', 0.1180990226041351, 0.9755790950580799)
('PP_0813_PP_1769_PP_4191-R1', 0.14146612383166055, 0.9367459724726824)
('PP_0368_PP_4191_PP_4192-R1', 0.10916526584610695, 0.9735275913249747)
('PP_0226_PP_0897_PP_5288-R2', 0.10381879142084728, 0.9804490812861448)
('PP_1319_PP_1457_PP_4192-R2', 0.09318304060921227, 0.980091339031148)
('PP_2213_PP_2471_PP_4266-R2', 0.09317985209738425, 0.9661418325939183)
('PP_3123_PP_4862_PP_5186-R2', 0.10480509013785291, 0.9827270112517912)
('PP_2137_PP_4116_PP_4187-R2', 0.10213653284224272, 0.977951366354064)
('PP_4189_PP_4550_PP_4862-R2', 0.1109497159286652, 0.9745202003955923)

In [5]:
df_output = pd.DataFrame(data)

# Filename for the output CSV file
filename = "DBTL1_mu_max.csv"

# Save DataFrame to CSV
df_output.to_csv(filename, index=False)

print(f"Data has been written to {filename}")

Data has been written to DBTL1_mu_max.csv
